In [1]:
from keras.models import load_model
import numpy as np

In [2]:
# Load the saved models
model1 = load_model('../models/alexnet_0_ultrasound')
model2 = load_model('../models/fcn_0_ultrasound')

In [4]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 225, 225, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 225, 225, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 113, 113, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 111, 111, 64)      18496     
                                                                 
 activation_1 (Activation)   (None, 111, 111, 64)      0

In [ ]:
# Make predictions on test data using both models
X_test = # load your test data here
predictions1 = model1.predict(X_test)
predictions2 = model2.predict(X_test)

# Combine predictions using voting
ensemble_predictions = np.round((predictions1 + predictions2) / 2)

# Evaluate accuracy of ensemble predictions
y_test = # load your test labels here
ensemble_accuracy = model1.evaluate(X_test, y_test, verbose=0)[1]
print("Ensemble accuracy: {:.2f}%".format(ensemble_accuracy * 100))
